<a href="https://colab.research.google.com/github/fresilva-ai/public.ai/blob/main/Deepfake_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Instalar dependencias
!pip install insightface opencv-python numpy onnxruntime
!pip install -U gdown

In [10]:
# Importar librerías necesarias
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
import onnxruntime as ort
from google.colab import files
import os
import shutil

In [20]:
# Configurar Colab para usar GPU
import torch
use_gpu = torch.cuda.is_available()
if use_gpu:
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    print("GPU activada para acelerar el procesamiento.")
else:
    providers = ['CPUExecutionProvider']
    print("GPU no disponible. Usando CPU.")

# Descargar modelo de análisis facial (buffalo_l)
face_model_url = "https://github.com/deepinsight/insightface/releases/download/v0.0.1/buffalo_l.zip"
model_dir = "buffalo_l"
if not os.path.exists(model_dir):
    print("Descargando modelo buffalo_l...")
    !wget -O buffalo_l.zip {face_model_url}
    !unzip buffalo_l.zip -d ./

# Inicializar el modelo de análisis facial
face_app = FaceAnalysis(name='buffalo_l', providers=providers)
face_app.prepare(ctx_id=0, det_size=(640, 640))

# Función para alinear rostros usando puntos clave
def align_faces(source_img, target_img, source_face, target_face):
    source_landmarks = source_face.landmark_2d_106
    target_landmarks = target_face.landmark_2d_106

    # Seleccionamos más puntos clave para mejor alineación (ojos, nariz y boca)
    points = [33, 36, 39, 42, 45, 48, 54]  # Nariz, ojos y extremos de la boca
    source_points = np.float32([source_landmarks[p] for p in points])
    target_points = np.float32([target_landmarks[p] for p in points])

    # Calcular la transformación afín
    M = cv2.estimateAffinePartial2D(source_points, target_points, method=cv2.LMEDS)[0]
    aligned_face = cv2.warpAffine(source_img, M, (target_img.shape[1], target_img.shape[0]))

    return aligned_face

# Función para ajustar la iluminación y el balance de color
def adjust_lighting(source_face, target_face):
    source_lab = cv2.cvtColor(source_face, cv2.COLOR_RGB2LAB)
    target_lab = cv2.cvtColor(target_face, cv2.COLOR_RGB2LAB)

    # Calcular la media de iluminación en el canal L
    l_mean_src = cv2.mean(source_lab[:,:,0])[0]
    l_mean_tgt = cv2.mean(target_lab[:,:,0])[0]

    l_scale = l_mean_tgt / l_mean_src
    source_lab[:,:,0] = np.clip(source_lab[:,:,0] * l_scale, 0, 255).astype(np.uint8)
    adjusted_face = cv2.cvtColor(source_lab, cv2.COLOR_LAB2RGB)

    return adjusted_face

# Función para intercambiar rostros con mejor fusión y adaptación de iluminación
def swap_faces(source_img, target_img):
    source_faces = face_app.get(source_img)
    target_faces = face_app.get(target_img)

    if len(source_faces) == 0 or len(target_faces) == 0:
        print("No se encontraron rostros en una de las imágenes.")
        return None

    result_img = target_img.copy()

    # Usar el primer rostro detectado de cada imagen
    source_face = source_faces[0]
    target_face = target_faces[0]

    # Alinear el rostro fuente al destino
    aligned_face = align_faces(source_img, target_img, source_face, target_face)

    # Extraer región del rostro destino
    target_box = target_face.bbox.astype(int)
    aligned_crop = aligned_face[target_box[1]:target_box[3], target_box[0]:target_box[2]]

    # Redimensionar el rostro alineado al tamaño del rostro destino
    resized_face = cv2.resize(aligned_crop, (target_box[2] - target_box[0], target_box[3] - target_box[1]))

    # Ajustar la iluminación para que coincida con el rostro destino
    adjusted_face = adjust_lighting(resized_face, target_img[target_box[1]:target_box[3], target_box[0]:target_box[2]])

    # Crear una máscara basada en detección de piel
    mask = np.zeros_like(target_img[target_box[1]:target_box[3], target_box[0]:target_box[2]], dtype=np.uint8)
    h, w, _ = mask.shape
    center = (w // 2, h // 2)
    cv2.circle(mask, center, min(h, w) // 2, (255, 255, 255), -1)

    # Aplicar clonación sin costuras
    blended_face = cv2.seamlessClone(adjusted_face, target_img[target_box[1]:target_box[3], target_box[0]:target_box[2]], mask, center, cv2.NORMAL_CLONE)
    result_img[target_box[1]:target_box[3], target_box[0]:target_box[2]] = blended_face

    return result_img

# Subir imágenes
print("Sube la imagen de la persona que quieres intercambiar (Fuente)")
uploaded_source = files.upload()
source_path = list(uploaded_source.keys())[0]

print("Sube la imagen de destino donde quieres poner el rostro")
uploaded_target = files.upload()
target_path = list(uploaded_target.keys())[0]

# Cargar imágenes
source_img = cv2.imread(source_path)
target_img = cv2.imread(target_path)

# Verificar si se cargaron correctamente
if source_img is None or target_img is None:
    print("Error al cargar las imágenes.")
else:
    # Convertir de BGR a RGB
    source_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)
    target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)

    # Realizar intercambio de rostros
    result_img = swap_faces(source_img, target_img)

    if result_img is not None:
        # Guardar y mostrar resultado
        result_path = "face_swapped_result.jpg"
        cv2.imwrite(result_path, cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))
        files.download(result_path)
        print("Intercambio de rostros completado. Descarga tu imagen.")

GPU activada para acelerar el procesamiento.
Descargando modelo buffalo_l...
--2025-02-11 01:07:46--  https://github.com/deepinsight/insightface/releases/download/v0.0.1/buffalo_l.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-11 01:07:47 ERROR 404: Not Found.

Archive:  buffalo_l.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of buffalo_l.zip or
        buffalo_l.zip.zip, and cannot find buffalo_l.zip.ZIP, period.
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 

Saving images (2).jpeg to images (2) (6).jpeg
Sube la imagen de destino donde quieres poner el rostro


Saving IMG_3911_Original.png to IMG_3911_Original (11).png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Intercambio de rostros completado. Descarga tu imagen.
